# X-ray Absorption Spectroscopy (XAS) Experiment at SCS

Author: Jun Zhu jun.zhu@xfel.eu

Created on Nov. 12, 2018

Copyright (C) European X-Ray Free-Electron Laser Facility GmbH.

All rights reserved.

![](misc/illustration.png)

In [ ]:
import os
import numpy as np
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt

from xas_processor import XasDigitizer, compute_absorption

%load_ext autoreload
%autoreload 2

In [ ]:
plt.rcParams['font.size'] = 12

In [ ]:
!ls /gpfs/exfel/exp/SCS/201831/p900048/raw

```py
class XasDigitizer(XasProcessor):
     def __init__(self, run_folder, channels=('D', 'B', 'C', 'A'), 
                  pulse_separation=880e-9, interleaved_mode=False):
         """Initialization.
         
         :param str run_folder: full path of the run folder.
         :param tuple channels: names of AdqDigitizer channels which connects to MCP1 to MCP4.
         :param float pulse_separation: pulse separation in a train, in s.
         :param bool interleaved_mode: the resolution is improved by a factor of two 
             in the interleaved mode. Default = False.
         """
```

In [ ]:
run_folder = "/gpfs/exfel/exp/SCS/201831/p900048/raw/r0313"

# instantiate an `XasDigitizer` object
xas = XasDigitizer(run_folder)
xas.info()

## Sanity check

```py
def plot_xgm_run(self, *, figsize=(8, 5.6)):
    """Plot the train resolved data from XGM.
    
    :param tuple figsize: figure size.
    """
```

In [ ]:
fig, (ax11, ax12, ax2) = xas.plot_xgm_run()

why negative pulse energies?

```py
def plot_xgm_train(self, *, index=0, train_id=None, figsize=(8, 5.6)):
    """Plot xgm measurement in a given train.

    :param int index: train index. Ignored if train_id is given.
    :param int train_id: train ID.
    :param tuple figsize: figure size.
    """
```

In [ ]:
fig, ax = xas.plot_xgm_train(index=10)

```py
def plot_digitizer_train(self, *, index=0, train_id=None, figsize=(8, 11.2),
                         x_min=None, x_max=None):
    """Plot digitizer signals in a given train.

    :param int index: train index. Ignored if train_id is given.
    :param int train_id: train ID.
    :param tuple figsize: figure size.
    :param int x_min: minimum sample ID.
    :param int x_max: maximum sample ID.
    """
```

In [ ]:
fig, axes = xas.plot_digitizer_train(index=200, x_min=8000, x_max=60000)

## Correlation between I0 and I1

```py
def process(self, pulse_id0, n_pulses, config=None):
    """Process the run data.

    :param int pulse_id0: first pulse ID.
    :param int n_pulses: number of pulses in a train.
    :param dict config: configuration for integrating of digitizer signal.
    """
```

In [ ]:
import time

t0 = time.time()

cfg = {"peak_start": 14590, "peak_width": 30, "background_end": 14580, "background_width": 500}
xas.process(30, 10, config=cfg)  # processing will take about 60 seconds for a 5-minute run

print("Processing time: {:.1f} seconds".format(time.time() - t0))

```py
@property
def data(self):
    """Get the pulse-resolved data in pandas.DataFrame.
   
    The data is not filtered! The signs of signals in MCP channels are flipped.
    """
```

In [ ]:
xas.data.head()

```py
def compute_total_absorption(self):
    """Compute absorption for all data.

    :return: total absorption dta in pandas.DataFrame with index being the 
        MCP channel name and columns being:
        - muA: absorption mean;
        - sigmaA: absorption standard deviation;
        - muI0: I0 mean;
        - sigmaI0: I0 standard deviation;
        - weight: sum of I0 values;
        - muI1: I1 mean;
        - sigmaI1: I1 standard deviation;
        - corr: correlation coefficient between I0 and I1;
        - count: number of data.
    """
```

In [ ]:
# get absorption data as pandas.DataFrame
total_absorption = xas.compute_total_absorption()

total_absorption.head()

```py
def plot_correlation(self, channel=None, *, figsize=(8, 6), 
                     ms=6, alpha=0.05, n_bins=20):
    """Generate correlation plots.

    :param str channel: MCP channel name, e.g. MCP1, for visualizing 
        a single channel with four plots, or None (default) for  
        visualizing all the channels with one plot each.  
        Case insensitive. 
    :param tuple figsize: figure size.
    :param int ms: marker size for the scatter plots.
    :param float alpha: transparency for the scatter plots.
    :param int n_bins: number of bins for the histogram plots.
    """
```

In [ ]:
fig, ax = xas.plot_correlation()

In [ ]:
fig, ax = xas.plot_correlation('mcp1')

In [ ]:
fig, ax = xas.plot_correlation('mcp2')

In [ ]:
fig, ax = xas.plot_correlation('mcp3')

In [ ]:
fig, ax = xas.plot_correlation('mcp4')

## Absorption spectrum

```py
def compute_spectrum(self, n_bins=20):
    """Calculate spectrum.

    :param int n_bins: number of energy bins.

    :return: spectrum data in pandas.DataFrame with index being the 
        energy bin range and columns being;
        - energy: central energy of each bin;
        - count: number of data points for each energy bin;
        - muXGM, muMCP1, muMCP2, muMCP3: intensity mean;
        - sigmaXGM, sigmaMCP1, sigmaMCP2, sigmaMCP3: standard deviation mean;
        - muA1, muA2, muA3: absorption mean;
        - sigmaA1, sigmaA2, sigmaA3: absorption standard deviation;
        - corrMCP1, corrMCP2, corrMCP3: correlation between XGM and MCP.
    """
```

In [ ]:
spectrum = xas.compute_spectrum()
spectrum

```py
def plot_spectrum(self, channel=None, *, figsize=(6, 4.5), capsize=4, 
                  n_bins=20, use_transmission=False):
    """Generate spectrum plots.

    :param str channel: MCP channel name, e.g. MCP1, for visualizing
        a single channel, or None (default) for visualizing MCP1-3 
        altogether. Case insensitive.
    :param tuple figsize: figure size.
    :param int capsize: cap size for the error bar.
    :param int n_bins: number of energy bins.
    :param bool use_transmission: False for plotting energy vs. 
        absorption, while True for plotting energy vs. I1. 
        Default = False
    """
```

In [ ]:
fig, ax = xas.plot_spectrum()

In [ ]:
fig, ax = xas.plot_spectrum(use_transmission=True)

In [ ]:
fig, ax = xas.plot_spectrum('mcp2')

In [ ]:
fig, ax = xas.plot_spectrum('mcp2', use_transmission=True)